In [1]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    JSONAgentOutputParser,
    ReActSingleInputOutputParser,
)
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun, tool
from langchain.tools.render import render_text_description_and_args, format_tool_to_openai_function
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder
from langchain.schema import ChatMessage, SystemMessage

from pymatgen.core.periodic_table import _pt_data

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert
)
from llamp.arxiv.agents import ArxivAgent

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)

# OPENAI_GPT_MODEL = "gpt-4-1106-preview"
OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"

In [2]:
import re

mp_llm = ChatOpenAI(
    temperature=0.2,
    model="gpt-4-1106-preview",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    max_retries=5,
    # streaming=True
)

llm = ChatOpenAI(
    temperature=0.7,
    model="gpt-4",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    # streaming=True
)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = [
    MPThermoExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElasticityExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPDielectricExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPMagnetismExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElectronicExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPSummaryExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPSynthesisExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    ArxivAgent(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    # arxiv,
    # wikipedia,
]

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages[0].prompt.template = re.sub(
    r"\s+", " ",
    """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    user to clarify their queries if needed. Please note that you don't have 
    direct control over MP but through multiple assistant agents to help you. 
    You need to provide complete context in the input for them to do their job.
    """).replace("\n", " ") + prompt.messages[0].prompt.template

prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm.bind(stop=["Observation"])
    # | map_reduce_chain  # TODO: Add map-reduce after LLM
    | JSONAgentOutputParser()
)


conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

agent_kwargs = {
    "handle_parsing_errors": True,
    "extra_prompt_messages": [
        MessagesPlaceholder(variable_name="chat_history"),
        # SystemMessage(content=re.sub(
        #     r"\s+", " ",
        #     """You are a helpful data-aware agent that can consult materials-related
        #     data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
        #     user to clarify their queries if needed. Please note that you don't have 
        #     direct control to MP but through multiple assistant agents to help you. 
        #     You need to provide complete context for them to do their job. 
        #     """).replace("\n", " ")
        # )
        ],
    # "early_stopping_method": 'generate',
    # "extra_prompt_messages": 
    # )
}

agent_executor = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=20,
    memory=conversational_memory,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
)

# agent_executor = initialize_agent(
#     tools=tools,
#     llm=llm,
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True,
#     max_iterations=5,
# )

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [3]:
agent_executor.invoke({
    "input": "Give me the formation energies of the following materials: Ag2O3, MgZn2, GaAs, GaN, SiC, BN, InSe, MoS2, BaTiO3, CsPbI3. If there are multiple values, please give me the smallest value.",
})



> Entering new AgentExecutor chain...


Action:
```
{
  "action": "MPThermoExpert",
  "action_input": "What are the formation energies of Ag2O3, MgZn2, GaAs, GaN, SiC, BN, InSe, MoS2, BaTiO3, CsPbI3?"
}
```

> Entering new AgentExecutor chain...


/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Action:
```
{
  "action": "search_materials_thermo__get",
  "action_input": {
    "formula": "Ag2O3,MgZn2,GaAs,GaN,SiC,BN,InSe,MoS2,BaTiO3,CsPbI3",
    "fields": "material_id,formula_pretty,formation_energy_per_atom"
  }
}
```{"formula": "Ag2O3,MgZn2,GaAs,GaN,SiC,BN,InSe,MoS2,BaTiO3,CsPbI3", "fields": "material_id,formula_pretty,formation_energy_per_atom"}


Retrieving ThermoDoc documents:   0%|          | 0/375 [00:00<?, ?it/s]

...

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Action:
```
{
  "action": "search_materials_thermo__get",
  "action_input": {
    "formula": "CsPbI3",
    "fields": "material_id,formula_pretty,formation_energy_per_atom"
  }
}
```{"formula": "CsPbI3", "fields": "material_id,formula_pretty,formation_energy_per_atom"}


Retrieving ThermoDoc documents:   0%|          | 0/7 [00:00<?, ?it/s]

[{'formula_pretty': 'CsPbI3', 'material_id': 'mp-540839', 'formation_energy_per_atom': -1.296129543586207}, {'formula_pretty': 'CsPbI3', 'material_id': 'mp-1120768', 'formation_energy_per_atom': -1.2796408769310343}, {'formula_pretty': 'CsPbI3', 'material_id': 'mp-540839', 'formation_energy_per_atom': -1.295464193931037}, {'formula_pretty': 'CsPbI3', 'material_id': 'mp-540839', 'formation_energy_per_atom': -1.2898320764999993}, {'formula_pretty': 'CsPbI3', 'material_id': 'mp-1069538', 'formation_energy_per_atom': -1.2704148499310342}, {'formula_pretty': 'CsPbI3', 'material_id': 'mp-1120768', 'formation_energy_per_atom': -1.280306226586207}, {'formula_pretty': 'CsPbI3', 'material_id': 'mp-1069538', 'formation_energy_per_atom': -1.271080199586207}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The formation energy per atom of CsPbI3 varies slightly depending on the specific material entry. Here are some values for different entries on Materials Project: -1.296 eV/atom (mp-5

{'input': 'Give me the formation energies of the following materials: Ag2O3, MgZn2, GaAs, GaN, SiC, BN, InSe, MoS2, BaTiO3, CsPbI3. If there are multiple values, please give me the smallest value.',
 'chat_history': [],
 'output': 'The smallest formation energies of the materials you asked for are as follows: Ag2O3: -0.3601 eV/atom, MgZn2: -0.1306 eV/atom, GaAs: -0.446 eV/atom, GaN: -0.052 eV/atom, SiC: -0.204 eV/atom, BN: -1.41 eV/atom, InSe: -0.742582154921875 eV/atom, MoS2: -1.202 eV/atom, BaTiO3: -3.493 eV/atom, CsPbI3: -1.296 eV/atom.'}

In [4]:
print(agent_executor.memory.chat_memory.messages[1].content)

The smallest formation energies of the materials you asked for are as follows: Ag2O3: -0.3601 eV/atom, MgZn2: -0.1306 eV/atom, GaAs: -0.446 eV/atom, GaN: -0.052 eV/atom, SiC: -0.204 eV/atom, BN: -1.41 eV/atom, InSe: -0.742582154921875 eV/atom, MoS2: -1.202 eV/atom, BaTiO3: -3.493 eV/atom, CsPbI3: -1.296 eV/atom.


In [5]:

llm_gpt = ChatOpenAI(
    temperature=0.7,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    # streaming=True
)

response = llm_gpt.invoke(agent_executor.memory.chat_memory.messages[0].content)


In [6]:
print(response.content)

Ag2O3: -2.63 eV/atom
MgZn2: -0.27 eV/atom
GaAs: -4.13 eV/atom
GaN: -3.51 eV/atom
SiC: -4.98 eV/atom
BN: -3.8 eV/atom
InSe: -0.89 eV/atom
MoS2: -3.31 eV/atom
BaTiO3: -4.45 eV/atom
CsPbI3: -2.32 eV/atom
